In [1]:
import pandas as pd
import os
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Constants, definitions etc

In [3]:
from src.utils.globals import *

Collecting all the individual series into a single dataset

In [4]:
def process_df(dfpath, df_program):
    df = pd.read_csv(dfpath)  
    if df.empty:
        return df  

    prows = ['sequence', 'anomaly', 'seqid']
    imurows = ['label', 'anomaly', 'seqid']

    #print(df_program)
    print(dfpath)

    df[imurows] = df_program.iloc[-1][prows]
    

    for _, row in df_program.iterrows():
        df.loc[(df['time'] >= row['time']) & (df['time'] <= row['time2']), 'label'] = row['sequence']

    return df


def process_folder(folder_path):
    paths = re.split(r'/|\\', folder_path)
    date = paths[1]
    time = paths[3]
    datetime = date + "|" + time

    df_program = pd.read_csv(folder_path + "\\program.csv")
    df_program['seqid'] = datetime
    df_program['time2'] = df_program['time'].shift(-1)
    
    df_imu = process_df(folder_path + "\\imu.csv", df_program)
    #df_motor = process_df(folder_path + "\\motor.csv", df_program)

    #return [df_imu, df_motor]
    return df_imu

def recursive_folder_traversal(current_path):
    items = os.listdir(current_path)
    
    subfolders = [item for item in items if os.path.isdir(os.path.join(current_path, item))]

    if not subfolders:
        yield process_folder(current_path)
    else:
        for subfolder in subfolders:
           yield from recursive_folder_traversal(os.path.join(current_path, subfolder))

imu_dfs = [dfi for dfi in recursive_folder_traversal(data_folder_path)]
df_imu = pd.concat(imu_dfs)
# #df_motor = pd.DataFrame()
# for dfi in recursive_folder_traversal(data_folder_path):

#     imu_dfs += [dfi]
#     #df_motor = pd.concat([df_motor, dfm])



data\2024.09.25\Athena\15.55.33\imu.csv
data\2024.09.25\Athena\15.56.59\imu.csv
data\2024.09.25\Athena\15.58.29\imu.csv
data\2024.09.25\Athena\15.59.59\imu.csv
data\2024.09.25\Athena\16.01.30\imu.csv
data\2024.09.25\Athena\16.03.00\imu.csv
data\2024.09.25\Athena\16.04.30\imu.csv
data\2024.09.25\Athena\16.06.01\imu.csv
data\2024.09.25\Athena\16.07.32\imu.csv
data\2024.09.25\Athena\16.09.02\imu.csv
data\2024.09.25\Athena\16.10.32\imu.csv
data\2024.09.25\Athena\16.12.03\imu.csv
data\2024.09.25\Athena\16.13.33\imu.csv
data\2024.09.25\Athena\16.15.03\imu.csv
data\2024.09.25\Athena\16.16.34\imu.csv
data\2024.09.25\Athena\16.18.04\imu.csv
data\2024.09.25\Athena\16.19.34\imu.csv
data\2024.09.25\Athena\16.21.05\imu.csv
data\2024.09.25\Athena\16.22.35\imu.csv
data\2024.09.25\Athena\16.24.05\imu.csv
data\2024.09.25\Athena\16.25.36\imu.csv
data\2024.09.25\Athena\16.27.06\imu.csv
data\2024.09.25\Athena\16.28.37\imu.csv
data\2024.09.25\Athena\16.30.07\imu.csv
data\2024.09.25\Athena\16.31.38\imu.csv


In [5]:
df_motor = pd.DataFrame()

Fixing mistake, prepreprocessing & saving the datasets

In [6]:
# # Removing data from the motor that does not exists.
# #df_motor = df_motor[df_motor['mid'] < 3].copy().reset_index()

# # Making a column with a single timevalue (called timeindex) for the 3 motors.
# df_motor['group'] = (df_motor['mid'] == 0).cumsum()
# df_timeindex = df_motor.groupby('group').agg(timeindex = ('time' , 'mean')).reset_index()
# df_motor = df_motor.merge(df_timeindex, on='group')

# # Dropping the columns that are unnecesary
# df_motor = df_motor.drop(columns=['freq','amp','offset','shift','phase'])

In [7]:
#Fixing the real column
df_imu['real'] = df_imu['real'].astype(dtype='float64')
df_imu = df_imu.fillna(0)

# Making a column with a single timevalue (called timeindex) for the 4 IMU types
df_imu['group'] = (df_imu['type'] == 'acceleration').cumsum()
df_timeindex = df_imu.groupby('group').agg(timeindex = ('time' , 'mean')).reset_index()
df_imu = df_imu.merge(df_timeindex, on='group')

Pivoting Motor

In [8]:
# df_pivot = df_motor.pivot_table(index=['seqid', 'timeindex'], 
#                                 columns='mid', 
#                                 values=['angle', 'output'])
# df_pivot.columns = [f'{col[0]}_{col[1]}' for col in df_pivot.columns]
# df_pivot = df_pivot.reset_index()

# df_motor_info = df_motor[df_motor['mid'] == 0][['seqid', 'timeindex', 'label', 'anomaly']]

# df_motor = df_pivot.merge(df_motor_info, on=['seqid', 'timeindex'])


Pivoting IMU

In [9]:
df_pivot = df_imu.pivot_table(index=['seqid', 'timeindex'], 
                                columns='type', 
                                values=['x', 'y', 'z', 'real'])
df_pivot.columns = [f'{col[1]}_{col[0]}' for col in df_pivot.columns]
df_pivot = df_pivot.reset_index()
df_pivot = df_pivot.drop(columns=['acceleration_real', 'gravity_real', 'gyroscope_real', 'magnetic_real'])

df_imu_info = df_imu[df_imu['type'] == 'acceleration'][['seqid', 'timeindex', 'label', 'anomaly']]

df_imu = df_pivot.merge(df_imu_info, on=['seqid', 'timeindex'])

In [10]:
df_imu.to_parquet(path_imu_raw)
#df_motor.to_parquet(path_motor_raw)

Cell to read the datasets instead when collecting in case collecting is already done.

In [11]:
#df_imu = pd.read_parquet(path_imu_raw)
#df_motor = pd.read_parquet(path_motor_raw)

Processing the input database such that all series start at a timeindex of 0

In [12]:
def normalize_time(df):
    tmp_df = df.groupby(['seqid']).agg(timeindex_min=('timeindex', 'min')).reset_index()
    df = df.merge(tmp_df, on=['seqid'])
    df['timeindex'] = df['timeindex'] - df['timeindex_min']
    df = df.drop(labels=["timeindex_min"], axis=1)
    df = df.reset_index(drop=True)
    return df

df_imu = normalize_time(df_imu)
#df_motor = normalize_time(df_motor)


Making timebin entries, such that the time-interval between measurements are a constant.

In [13]:
def normalize_timeinterval(df):
    df = df.sort_values(by=['seqid', 'timeindex'])
    time_dt = np.median(df['timeindex'].diff().dropna())
    df['timeindex_bin'] = np.round(df['timeindex'] / time_dt).astype(int)
    return df

df_imu = normalize_timeinterval(df_imu)
#df_motor = normalize_timeinterval(df_motor)


Combining Duplicates

In [14]:
imu_cols_data = ['timeindex', 'rotation_real', 'acceleration_x', 'gravity_x',
       'gyroscope_x', 'magnetic_x', 'rotation_x', 'acceleration_y',
       'gravity_y', 'gyroscope_y', 'magnetic_y', 'rotation_y',
       'acceleration_z', 'gravity_z', 'gyroscope_z', 'magnetic_z',
       'rotation_z']

df_imu = df_imu.groupby(['seqid', 'timeindex_bin']).agg({key : 'mean' for key in imu_cols_data} | {key : 'first' for key in ['label', 'anomaly']})
df_imu = df_imu.reset_index()


In [15]:
# motor_cols_data = ['timeindex', 'angle_0', 'angle_1', 'angle_2', 'output_0', 'output_1', 'output_2']

# df_motor = df_motor.groupby(['seqid', 'timeindex_bin']).agg({key : 'mean' for key in motor_cols_data} | {key : 'first' for key in ['label', 'anomaly']})
# df_motor = df_motor.reset_index()

Interpolation

In [16]:
#Interpolation
def fill_and_interpolate(group, datacols):
    infocols = ['seqid', 'label', 'anomaly']

    # Add rows that do not have values
    full_range = pd.DataFrame({'timeindex_bin': range(0, group['timeindex_bin'].max() + 1)})
    group = pd.merge(full_range, group, how='left', on=['timeindex_bin'])
    group.sort_values('timeindex_bin')

    # Fill & Interpolate
    group[datacols] = group[datacols].interpolate(method='linear')
    group[infocols] = group[infocols].ffill().bfill()

    return group

df_imu = df_imu.groupby('seqid').apply(lambda x : fill_and_interpolate(x, imu_cols_data)).reset_index(drop=True)
#df_motor = df_motor.groupby('seqid').apply(lambda x : fill_and_interpolate(x, motor_cols_data)).reset_index(drop=True)
    

# Min Max Normalization 

In [17]:
# cols = [f'output_{x}' for x in range(3)]
# df_motor[cols] = MinMaxScaler((-1,1)).fit_transform(df_motor[cols])

# for sensor, axes in imu_sensor_axes:
#     columns = [f'{sensor}_{axis}' for axis in axes]
#     values = df_imu[columns].values

#     scaler = MinMaxScaler((-1, 1)).fit(values.reshape(-1, 1))
#     for col in columns:
#         df_imu[col] = scaler.transform(df_imu[col].values.reshape(-1, 1))

Writing DF to path_out

In [18]:
df_imu.to_parquet(path_imu)
#df_motor.to_parquet(path_motor)

# Creation of result databases

In [19]:
df_series_results = df_imu.groupby('seqid').agg(anomaly=('anomaly', 'first')).reset_index()
df_series_results.to_parquet(path_series_results)

In [20]:
df_imu_results = df_imu.loc[:,['seqid', 'timeindex_bin', 'anomaly', 'label']]
df_imu_results.to_parquet(path_imu_results)

#df_motor_results = df_motor.loc[:,['seqid', 'timeindex_bin', 'anomaly', 'label']]
#df_motor_results.to_parquet(path_motor_results)